In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/cnn-processed-2-7/clean_chunk_2.csv
/kaggle/input/cnn-processed-2-7/chunk_2.csv


In [2]:
!pip3 install skorch torchview torchinfo transformers datasets evaluate rouge_score accelerate -U

  Preparing metadata (setup.py) ... - done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.4/239.4 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 13.1 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=f5097e526a51bc4501363d515fe69b35fdfd0feb140eb5ce37226ad940f5dcd8
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.30.1
    Uninstalling accelerate-0.30.1:
      Successfully uninstalled accelerate-0.30.1


In [3]:
import torch
import numpy as np
from torchview import draw_graph
from torchinfo import summary
import pandas as pd
import matplotlib.pyplot as plt
from timeit import default_timer as timer
import warnings
import evaluate
warnings.filterwarnings("ignore") 

2024-06-07 16:59:20.101004: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-07 16:59:20.101104: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-07 16:59:20.232441: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
from transformers import pipeline, AutoTokenizer


summarizer2 = pipeline("summarization", model="Falconsai/text_summarization", device=0) 



config.json:   0%|          | 0.00/1.49k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [5]:
data = pd.read_csv('/kaggle/input/cnn-processed-2-7/chunk_2.csv')
data = data.head(7000)  # Limit to the first 7,000 samples
articles = data['article'].tolist()

In [6]:
from rouge_score import rouge_scorer
def compute_rouge_scores(reference_summary, generated_summary):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(reference_summary, generated_summary)
    return scores



In [7]:
generated_summaries = []
rouge_scores_list = []
start_time = timer()
for index, row in data.iterrows():
    print(f"Processing row {index+1}/{len(data)}")
    article = row['article']
    reference_summary = row['summary']
    generated_summary = summarizer2(article, num_beams =2, min_length = 20, max_length=160, truncation=True)[0]['summary_text']
    
    # Compute ROUGE scores
    rouge_scores = compute_rouge_scores(reference_summary, generated_summary)
    generated_summaries.append(generated_summary)
    rouge_scores_list.append(rouge_scores)
end_time = timer()
total_time = end_time - start_time
print(f"Total time: {total_time:.4f} seconds")
data['generated_summary'] = generated_summaries
data['rouge_scores'] = rouge_scores_list

Processing row 1/7000
Processing row 2/7000
Processing row 3/7000
Processing row 4/7000
Processing row 5/7000
Processing row 6/7000
Processing row 7/7000
Processing row 8/7000
Processing row 9/7000
Processing row 10/7000


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Processing row 11/7000
Processing row 12/7000
Processing row 13/7000
Processing row 14/7000
Processing row 15/7000
Processing row 16/7000
Processing row 17/7000
Processing row 18/7000
Processing row 19/7000
Processing row 20/7000
Processing row 21/7000
Processing row 22/7000
Processing row 23/7000
Processing row 24/7000
Processing row 25/7000
Processing row 26/7000
Processing row 27/7000
Processing row 28/7000
Processing row 29/7000
Processing row 30/7000
Processing row 31/7000
Processing row 32/7000
Processing row 33/7000
Processing row 34/7000
Processing row 35/7000
Processing row 36/7000
Processing row 37/7000
Processing row 38/7000
Processing row 39/7000
Processing row 40/7000
Processing row 41/7000
Processing row 42/7000
Processing row 43/7000
Processing row 44/7000
Processing row 45/7000
Processing row 46/7000
Processing row 47/7000
Processing row 48/7000
Processing row 49/7000
Processing row 50/7000
Processing row 51/7000
Processing row 52/7000
Processing row 53/7000
Processing 

Your max_length is set to 160, but your input_length is only 150. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=75)


Processing row 853/7000
Processing row 854/7000
Processing row 855/7000
Processing row 856/7000
Processing row 857/7000
Processing row 858/7000
Processing row 859/7000
Processing row 860/7000
Processing row 861/7000
Processing row 862/7000
Processing row 863/7000
Processing row 864/7000
Processing row 865/7000
Processing row 866/7000
Processing row 867/7000
Processing row 868/7000
Processing row 869/7000
Processing row 870/7000
Processing row 871/7000
Processing row 872/7000
Processing row 873/7000
Processing row 874/7000
Processing row 875/7000
Processing row 876/7000
Processing row 877/7000
Processing row 878/7000
Processing row 879/7000
Processing row 880/7000
Processing row 881/7000
Processing row 882/7000
Processing row 883/7000
Processing row 884/7000
Processing row 885/7000
Processing row 886/7000
Processing row 887/7000
Processing row 888/7000
Processing row 889/7000
Processing row 890/7000
Processing row 891/7000
Processing row 892/7000
Processing row 893/7000
Processing row 8

...

Processing row 3745/7000
Processing row 3746/7000
Processing row 3747/7000
Processing row 3748/7000
Processing row 3749/7000
Processing row 3750/7000
Processing row 3751/7000
Processing row 3752/7000
Processing row 3753/7000
Processing row 3754/7000
Processing row 3755/7000
Processing row 3756/7000
Processing row 3757/7000
Processing row 3758/7000
Processing row 3759/7000
Processing row 3760/7000
Processing row 3761/7000
Processing row 3762/7000
Processing row 3763/7000
Processing row 3764/7000
Processing row 3765/7000
Processing row 3766/7000
Processing row 3767/7000
Processing row 3768/7000
Processing row 3769/7000
Processing row 3770/7000
Processing row 3771/7000
Processing row 3772/7000
Processing row 3773/7000
Processing row 3774/7000
Processing row 3775/7000
Processing row 3776/7000
Processing row 3777/7000
Processing row 3778/7000
Processing row 3779/7000
Processing row 3780/7000
Processing row 3781/7000
Processing row 3782/7000
Processing row 3783/7000
Processing row 3784/7000


Your max_length is set to 160, but your input_length is only 65. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=32)


Processing row 4701/7000
Processing row 4702/7000
Processing row 4703/7000
Processing row 4704/7000
Processing row 4705/7000
Processing row 4706/7000
Processing row 4707/7000
Processing row 4708/7000
Processing row 4709/7000
Processing row 4710/7000
Processing row 4711/7000
Processing row 4712/7000
Processing row 4713/7000
Processing row 4714/7000
Processing row 4715/7000
Processing row 4716/7000
Processing row 4717/7000
Processing row 4718/7000
Processing row 4719/7000
Processing row 4720/7000
Processing row 4721/7000
Processing row 4722/7000
Processing row 4723/7000
Processing row 4724/7000
Processing row 4725/7000
Processing row 4726/7000
Processing row 4727/7000
Processing row 4728/7000
Processing row 4729/7000
Processing row 4730/7000
Processing row 4731/7000
Processing row 4732/7000
Processing row 4733/7000
Processing row 4734/7000
Processing row 4735/7000
Processing row 4736/7000
Processing row 4737/7000
Processing row 4738/7000
Processing row 4739/7000
Processing row 4740/7000


Your max_length is set to 160, but your input_length is only 139. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=69)


Processing row 5367/7000
Processing row 5368/7000
Processing row 5369/7000
Processing row 5370/7000
Processing row 5371/7000
Processing row 5372/7000
Processing row 5373/7000
Processing row 5374/7000
Processing row 5375/7000
Processing row 5376/7000
Processing row 5377/7000
Processing row 5378/7000
Processing row 5379/7000
Processing row 5380/7000
Processing row 5381/7000
Processing row 5382/7000
Processing row 5383/7000
Processing row 5384/7000
Processing row 5385/7000
Processing row 5386/7000
Processing row 5387/7000
Processing row 5388/7000
Processing row 5389/7000
Processing row 5390/7000
Processing row 5391/7000
Processing row 5392/7000
Processing row 5393/7000
Processing row 5394/7000
Processing row 5395/7000
Processing row 5396/7000
Processing row 5397/7000
Processing row 5398/7000
Processing row 5399/7000
Processing row 5400/7000
Processing row 5401/7000
Processing row 5402/7000
Processing row 5403/7000
Processing row 5404/7000
Processing row 5405/7000
Processing row 5406/7000


In [8]:
data.to_csv('Output.csv', index=True)